# 総合添削問題

機械学習における画像認識では、画像データとその画像に紐づけられたラベルの組み合わせが大量に必要となります。しかしながら、機械学習の入力とするには十分な数の画像とラベルの組み合わせを用意する事は、コストがかかるため困難なことであります。
そこで、データの個数を十分量に増やす際に行われるテクニックとして、**画像の水増し**があります。

画像の水増しといっても、ただ単にデータをコピーして量を増やすだけでは意味がありません。<br>
そこで、例えば画像を反転したり、ずらしたりして新たなデータを作り出します。ここでは、主にChapter3で学んだ様々な関数を駆使して画像を水増しする関数を作成してもらいます。

#### 問題

- 画像を受け取ったら5つの方法で加工した画像データを作成し、まとめて配列にして返す関数`scratch_image`関数を作成して下さい。
```python
def scratch_image(img, flip=True, thr=True, filt=True, resize=True, erode=True):
    """
    flip は画像の左右反転
    thr  は閾値処理
    filt はぼかし
    resizeはモザイク
    erode は収縮
        をするorしないを指定している
        
    imgの型はOpenCVのcv2.read()によって読み込まれた画像データの型
    
    水増しした画像データを配列にまとめて返す
    """
```

- 加工の方法は重ねあわせます。たとえば`flip=True, thr=True, filt=False, resize=False, erode=False`ならば画像の反転と閾値処理を行うので
    1. オリジナルの画像
    2. 左右反転した画像
    3. 閾値処理を行った画像
    4. 左右反転して閾値処理した画像
    
  と、四枚の画像データが配列にまとめられて返されます。
- 全て`True`ならば $2^5 = 32$ 枚の画像データが返されます。
- 作成した`scratch_image`関数を使ってcleansing_dataフォルダ内の画像データ（`cat_sample.png`）を水増しし、`scratch_images`フォルダに保存して下さい。
- 各手法の仕様は以下のようにして下さい。
    1. 反転: 左右で反転
    2. 閾値処理: 閾値100, しきい値より大きい値はそのまま、小さい値は0にする
    3. ぼかし: 自分自身のまわりの$5 \times 5$個のピクセルを用いる
    4. モザイク: 解像度を$1/5$にする
    5. 収縮: 自身を囲む8ピクセルを用いる

<div style="text-align: center;">
    <span style="font-size: 120%">flipとthrのみTrueだった場合の例</span>
</div>
<img src = "https://aidemyexstorage.blob.core.windows.net/aidemycontents/1539254347176308.png	" width="800px">

In [70]:
import os

import numpy as np
import matplotlib.pyplot as plt
import cv2
import itertools 
import shutil

# 
def scratch_image(img, flip=True, thr=True, filt=True, resize=True, erode=True):
    """
    flip は画像の左右反転
    thr  は閾値処理
    filt はぼかし
    resizeはモザイク
    erode は縮小
        をするorしないを指定している

    imgの型はOpenCVのcv2.read()によって読み込まれた画像データの型

    水増しした画像データを配列にまとめて返す
    """
    # ----------------------------ここから書いて下さい----------------------------
    flags = []
    processes = []
    
    
    if flip == True:
        processes.append(lambda img: cv2.flip(img, 1)) # flip, 左右反転=Y軸で反転 第二引数 1:Y軸指定
    if thr == True:
        processes.append(lambda img: cv2.threshold(img, 100, 255, cv2.THRESH_TOZERO)[1]) # 閾値100に満たない場合は0
    if filt == True:
        processes.append(lambda img:  cv2.GaussianBlur(img, (5, 5), 0))
    if resize == True:
        processes.append(lambda img: cv2.resize(img, dsize=None, fx=0.2, fy=0.2))
    if erode == True:
        processes.append(lambda img: cv2.erode(img, np.array([[1,1,1],[1,0,1],[1,1,1]],np.uint8)))

    patterns = []
    flags = range(len(processes))
    # 指定されたフラグの組み合わせを生成
    for r in flags:
        patterns.extend(list(itertools .combinations(flags, r+1)))

    images = [img]
    for pattern in patterns:
        img_after = np.copy(img)

        for process_id in pattern:
            img_after = processes[process_id](img_after)
        
        images.append(img_after)

    return  images
    # ----------------------------ここまで書いて下さい----------------------------

print ("始め")

# 画像の読み込み
cat_img = cv2.imread("cleansing_data/cat_sample.jpg")

# 画像の水増し
print ("水増し")
scratch_cat_images = scratch_image(cat_img, True, True, True, False, False)

# 画像を保存するフォルダーがあれば一旦削除
if os.path.exists("scratch_images"):
    shutil.rmtree("scratch_images")

# 画像を保存するフォルダーを作成
if not os.path.exists("scratch_images"):
    os.mkdir("scratch_images")

print ("出力中")
for num, im in enumerate(scratch_cat_images):
    # まず保存先のディレクトリ"scratch_images/"を指定、番号を付けて保存
    cv2.imwrite("scratch_images/" + ("%02d" % (num)) + ".jpg" ,im) 
    
print ("終わり")


始め
水増し
出力中
終わり


#### ヒント

- まずはChapter3を参考にして`cv2`のメソッドを記述します。その際に必要となるデータ（画像のサイズなど）も用意します。
- リスト内包表記を駆使することで、コードが簡略化できます。また、長い処理は`lambda`を駆使してまとめましょう。
- (発展)水増しに用いる関数を`lambda`で記述し、`np.array`に格納するといいです。<br>
例えば上下反転する関数は
```python
lambda x: cv2.flip(x, 0)
```
と書けます。これを配列`arr`に入れることで
```python
arr[0](image)
```
と、indexを指定するだけで`arr`中の所定の関数を使うことができます。
- (発展)関数を格納した配列と`flip, thr, filt, resize, erode`を活用して加工に使う関数を取得しましょう。

##  解答例

### 解説

添削課題の提出は以下のアドレスから提出いただきますようお願いします。<br>

https://goo.gl/forms/fW7CAspZMwHuWuqk2<br><br>
以下のアドレスからアンケートにご協力頂きたく存じます。<br>
ご回答のほど、よろしくお願いいたします。

https://goo.gl/forms/WHjJQYeodIndRvyz2